# Information Retrieval 1#
## Assignment 2: Retrieval models [100 points] ##

In this assignment you will get familiar with basic and advanced information retrieval concepts. You will implement different information retrieval ranking models and evaluate their performance.

We provide you with a Indri index. To query the index, you'll use a Python package ([pyndri](https://github.com/cvangysel/pyndri)) that allows easy access to the underlying document statistics.

For evaluation you'll use the [TREC Eval](https://github.com/usnistgov/trec_eval) utility, provided by the National Institute of Standards and Technology of the United States. TREC Eval is the de facto standard way to compute Information Retrieval measures and is frequently referenced in scientific papers.

This is a **groups-of-three assignment**, the deadline is **Wednesday, January 31st**. Code quality, informative comments and convincing analysis of the results will be considered when grading. Submission should be done through blackboard, questions can be asked on the course [Piazza](piazza.com/university_of_amsterdam/spring2018/52041inr6y/home).

### Technicalities (must-read!) ###

The assignment directory is organized as follows:
   * `./assignment.ipynb` (this file): the description of the assignment.
   * `./index/`: the index we prepared for you.
   * `./ap_88_90/`: directory with ground-truth and evaluation sets:
      * `qrel_test`: test query relevance collection (**test set**).
      * `qrel_validation`: validation query relevance collection (**validation set**).
      * `topics_title`: semicolon-separated file with query identifiers and terms.

You will need the following software packages (tested with Python 3.5 inside [Anaconda](https://conda.io/docs/user-guide/install/index.html)):
   * Python 3.5 and Jupyter
   * Indri + Pyndri (Follow the installation instructions [here](https://github.com/nickvosk/pyndri/blob/master/README.md))
   * gensim [link](https://radimrehurek.com/gensim/install.html)
   * TREC Eval [link](https://github.com/usnistgov/trec_eval)

### TREC Eval primer ###
The TREC Eval utility can be downloaded and compiled as follows:

    git clone https://github.com/usnistgov/trec_eval.git
    cd trec_eval
    make

TREC Eval computes evaluation scores given two files: ground-truth information regarding relevant documents, named *query relevance* or *qrel*, and a ranking of documents for a set of queries, referred to as a *run*. The *qrel* will be supplied by us and should not be changed. For every retrieval model (or combinations thereof) you will generate a run of the top-1000 documents for every query. The format of the *run* file is as follows:

    $query_identifier Q0 $document_identifier $rank_of_document_for_query $query_document_similarity $run_identifier
    
where
   * `$query_identifier` is the unique identifier corresponding to a query (usually this follows a sequential numbering).
   * `Q0` is a legacy field that you can ignore.
   * `$document_identifier` corresponds to the unique identifier of a document (e.g., APXXXXXXX where AP denotes the collection and the Xs correspond to a unique numerical identifier).
   * `$rank_of_document_for_query` denotes the rank of the document for the particular query. This field is ignored by TREC Eval and is only maintained for legacy support. The ranks are computed by TREC Eval itself using the `$query_document_similarity` field (see next). However, it remains good practice to correctly compute this field.
   * `$query_document_similarity` is a score indicating the similarity between query and document where a higher score denotes greater similarity.
   * `$run_identifier` is an identifier of the run. This field is for your own convenience and has no purpose beyond bookkeeping.
   
For example, say we have two queries: `Q1` and `Q2` and we rank three documents (`DOC1`, `DOC2`, `DOC3`). For query `Q1`, we find the following similarity scores `score(Q1, DOC1) = 1.0`, `score(Q1, DOC2) = 0.5`, `score(Q1, DOC3) = 0.75`; and for `Q2`: `score(Q2, DOC1) = -0.1`, `score(Q2, DOC2) = 1.25`, `score(Q1, DOC3) = 0.0`. We can generate run using the following snippet:

In [2]:
import logging
import sys
import os

def write_run(model_name, data, out_f,
              max_objects_per_query=sys.maxsize,
              skip_sorting=False):
    """
    Write a run to an output file.
    Parameters:
        - model_name: identifier of run.
        - data: dictionary mapping topic_id to object_assesments;
            object_assesments is an iterable (list or tuple) of
            (relevance, object_id) pairs.
            The object_assesments iterable is sorted by decreasing order.
        - out_f: output file stream.
        - max_objects_per_query: cut-off for number of objects per query.
    """
    for subject_id, object_assesments in data.items():
        if not object_assesments:
            logging.warning('Received empty ranking for %s; ignoring.',
                            subject_id)

            continue

        # Probe types, to make sure everything goes alright.
        # assert isinstance(object_assesments[0][0], float) or \
        #     isinstance(object_assesments[0][0], np.float32)
        assert isinstance(object_assesments[0][1], str) or \
            isinstance(object_assesments[0][1], bytes)

        if not skip_sorting:
            object_assesments = sorted(object_assesments, reverse=True)

        if max_objects_per_query < sys.maxsize:
            object_assesments = object_assesments[:max_objects_per_query]

        if isinstance(subject_id, bytes):
            subject_id = subject_id.decode('utf8')

        for rank, (relevance, object_id) in enumerate(object_assesments):
            if isinstance(object_id, bytes):
                object_id = object_id.decode('utf8')

            out_f.write(
                '{subject} Q0 {object} {rank} {relevance} '
                '{model_name}\n'.format(
                    subject=subject_id,
                    object=object_id,
                    rank=rank + 1,
                    relevance=relevance,
                    model_name=model_name))
            
# The following writes the run to standard output.
# In your code, you should write the runs to local
# storage in order to pass them to trec_eval.
write_run(
    model_name='example',
    data={
        'Q1': ((1.0, 'DOC1'), (0.5, 'DOC2'), (0.75, 'DOC3')),
        'Q2': ((-0.1, 'DOC1'), (1.25, 'DOC2'), (0.0, 'DOC3')),
    },
    out_f=sys.stdout,
    max_objects_per_query=1000)

Q2 Q0 DOC2 1 1.25 example
Q2 Q0 DOC3 2 0.0 example
Q2 Q0 DOC1 3 -0.1 example
Q1 Q0 DOC1 1 1.0 example
Q1 Q0 DOC3 2 0.75 example
Q1 Q0 DOC2 3 0.5 example


Now, imagine that we know that `DOC1` is relevant and `DOC3` is non-relevant for `Q1`. In addition, for `Q2` we only know of the relevance of `DOC3`. The query relevance file looks like:

    Q1 0 DOC1 1
    Q1 0 DOC3 0
    Q2 0 DOC3 1
    
We store the run and qrel in files `example.run` and `example.qrel` respectively on disk. We can now use TREC Eval to compute evaluation measures. In this example, we're only interested in Mean Average Precision and we'll only show this below for brevity. However, TREC Eval outputs much more information such as NDCG, recall, precision, etc.

    $ trec_eval -m all_trec -q example.qrel example.run | grep -E "^map\s"
    > map                   	Q1	1.0000
    > map                   	Q2	0.5000
    > map                   	all	0.7500
    
Now that we've discussed the output format of rankings and how you can compute evaluation measures from these rankings, we'll now proceed with an overview of the indexing framework you'll use.

### Pyndri primer ###
For this assignment you will use [Pyndri](https://github.com/cvangysel/pyndri) [[1](https://arxiv.org/abs/1701.00749)], a python interface for [Indri](https://www.lemurproject.org/indri.php). We have indexed the document collection and you can query the index using Pyndri. We will start by giving you some examples of what Pyndri can do:

First we read the document collection index with Pyndri:

In [3]:
import pyndri

index = pyndri.Index('index/')

The loaded index can be used to access a collection of documents in an easy manner. We'll give you some examples to get some idea of what it can do, it is up to you to figure out how to use it for the remainder of the assignment.

First let's look at the number of documents, since Pyndri indexes the documents using incremental identifiers we can simply take the lowest index and the maximum document and consider the difference:

In [4]:
print("There are %d documents in this collection." % (index.maximum_document() - index.document_base()))

There are 164597 documents in this collection.


Let's take the first document out of the collection and take a look at it:

In [5]:
example_document = index.document(index.document_base())
print(example_document)

('AP890425-0001', (1360, 192, 363, 0, 880, 0, 200, 0, 894, 412, 92160, 3, 192, 0, 363, 34, 1441, 0, 174134, 0, 200, 0, 894, 412, 2652, 0, 810, 107, 49, 4903, 420, 0, 1, 48, 35, 489, 0, 35, 687, 192, 243, 0, 249311, 1877, 0, 1651, 1174, 0, 2701, 117, 412, 0, 810, 391, 245233, 1225, 5838, 16, 0, 233156, 3496, 0, 393, 17, 0, 2435, 4819, 930, 0, 0, 200, 0, 894, 0, 22, 398, 145, 0, 3, 271, 115, 0, 1176, 2777, 292, 0, 725, 192, 0, 0, 50046, 0, 1901, 1130, 0, 192, 0, 408, 0, 243779, 0, 0, 553, 192, 0, 363, 0, 3747, 0, 0, 0, 0, 1176, 0, 1239, 0, 0, 1115, 17, 0, 0, 585, 192, 1963, 0, 0, 412, 54356, 0, 773, 0, 0, 0, 192, 0, 0, 1130, 0, 363, 0, 545, 192, 0, 1174, 1901, 1130, 0, 4, 398, 145, 39, 0, 577, 0, 355, 0, 491, 0, 6025, 0, 0, 193156, 88, 34, 437, 0, 0, 1852, 0, 828, 0, 1588, 0, 0, 0, 2615, 0, 0, 107, 49, 420, 0, 0, 190, 7, 714, 2701, 0, 237, 192, 157, 0, 412, 34, 437, 0, 0, 200, 6025, 26, 0, 0, 0, 0, 363, 0, 22, 398, 145, 0, 200, 638, 126222, 6018, 0, 880, 0, 0, 161, 0, 0, 319, 894, 2701, 

Here we see a document consists of two things, a string representing the external document identifier and an integer list representing the identifiers of words that make up the document. Pyndri uses integer representations for words or terms, thus a token_id is an integer that represents a word whereas the token is the actual text of the word/term. Every id has a unique token and vice versa with the exception of stop words: words so common that there are uninformative, all of these receive the zero id.

To see what some ids and their matching tokens we take a look at the dictionary of the index:

In [6]:
token2id, id2token, _ = index.get_dictionary()
print(list(id2token.items())[:15])

[(1, 'new'), (2, 'percent'), (3, 'two'), (4, '1'), (5, 'people'), (6, 'million'), (7, '000'), (8, 'government'), (9, 'president'), (10, 'years'), (11, 'state'), (12, '2'), (13, 'states'), (14, 'three'), (15, 'time')]


Using this dictionary we can see the tokens for the (non-stop) words in our example document:

In [7]:
print([id2token[word_id] for word_id in example_document[1] if word_id > 0])

['52', 'students', 'arrested', 'takeover', 'university', 'massachusetts', 'building', 'fifty', 'two', 'students', 'arrested', 'tuesday', 'evening', 'occupying', 'university', 'massachusetts', 'building', 'overnight', 'protest', 'defense', 'department', 'funded', 'research', 'new', 'york', 'city', 'thousands', 'city', 'college', 'students', 'got', 'unscheduled', 'holiday', 'demonstrators', 'occupied', 'campus', 'administration', 'building', 'protest', 'possible', 'tuition', 'increases', 'prompting', 'officials', 'suspend', 'classes', '60', 'police', 'riot', 'gear', 'arrived', 'university', 'massachusetts', '5', 'p', 'm', 'two', 'hours', 'later', 'bus', 'drove', 'away', '29', 'students', 'camped', 'memorial', 'hall', 'students', 'charged', 'trespassing', '23', 'students', 'arrested', 'lying', 'bus', 'prevent', 'leaving', 'police', '300', 'students', 'stood', 'building', 'chanting', 'looking', 'students', 'hall', 'arrested', '35', 'students', 'occupied', 'memorial', 'hall', '1', 'p', 'm',

The reverse can also be done, say we want to look for news about the "University of Massachusetts", the tokens of that query can be converted to ids using the reverse dictionary:

In [8]:
query_tokens = index.tokenize("University of Massachusetts")
print("Query by tokens:", query_tokens)
query_id_tokens = [token2id.get(query_token,0) for query_token in query_tokens]
print("Query by ids with stopwords:", query_id_tokens)
query_id_tokens = [word_id for word_id in query_id_tokens if word_id > 0]
print("Query by ids without stopwords:", query_id_tokens)

Query by tokens: ['university', '', 'massachusetts']
Query by ids with stopwords: [200, 0, 894]
Query by ids without stopwords: [200, 894]


Naturally we can now match the document and query in the id space, let's see how often a word from the query occurs in our example document:

In [9]:
matching_words = sum([True for word_id in example_document[1] if word_id in query_id_tokens])
print("Document %s has %d word matches with query: \"%s\"." % (example_document[0], matching_words, ' '.join(query_tokens)))
print("Document %s and query \"%s\" have a %.01f%% overlap." % (example_document[0], ' '.join(query_tokens),matching_words/float(len(example_document[1]))*100))

Document AP890425-0001 has 13 word matches with query: "university  massachusetts".
Document AP890425-0001 and query "university  massachusetts" have a 2.5% overlap.


While this is certainly not everything Pyndri can do, it should give you an idea of how to use it. Please take a look at the [examples](https://github.com/cvangysel/pyndri) as it will help you a lot with this assignment.

**CAUTION**: Avoid printing out the whole index in this Notebook as it will generate a lot of output and is likely to corrupt the Notebook.

### Parsing the query file
You can parse the query file (`ap_88_89/topics_title`) using the following snippet:

In [10]:
import collections
import io
import logging
import sys
import time

def parse_topics(file_or_files,
                 max_topics=sys.maxsize, delimiter=';'):
    assert max_topics >= 0 or max_topics is None

    topics = collections.OrderedDict()

    if not isinstance(file_or_files, list) and \
            not isinstance(file_or_files, tuple):
        if hasattr(file_or_files, '__iter__'):
            file_or_files = list(file_or_files)
        else:
            file_or_files = [file_or_files]

    for f in file_or_files:
        assert isinstance(f, io.IOBase)

        for line in f:
            assert(isinstance(line, str))

            line = line.strip()

            if not line:
                continue

            topic_id, terms = line.split(delimiter, 1)

            if topic_id in topics and (topics[topic_id] != terms):
                    logging.error('Duplicate topic "%s" (%s vs. %s).',
                                  topic_id,
                                  topics[topic_id],
                                  terms)

            topics[topic_id] = terms

            if max_topics > 0 and len(topics) >= max_topics:
                break

    return topics

with open('./ap_88_89/topics_title', 'r') as f_topics:
    print(parse_topics([f_topics]))

OrderedDict([('51', 'Airbus Subsidies'), ('52', 'South African Sanctions'), ('53', 'Leveraged Buyouts'), ('54', 'Satellite Launch Contracts'), ('55', 'Insider Trading'), ('56', 'Prime (Lending) Rate Moves, Predictions'), ('57', 'MCI'), ('58', 'Rail Strikes'), ('59', 'Weather Related Fatalities'), ('60', 'Merit-Pay vs. Seniority'), ('61', 'Israeli Role in Iran-Contra Affair'), ('62', "Military Coups D'etat"), ('63', 'Machine Translation'), ('64', 'Hostage-Taking'), ('65', 'Information Retrieval Systems'), ('66', 'Natural Language Processing'), ('67', 'Politically Motivated Civil Disturbances'), ('68', 'Health Hazards from Fine-Diameter Fibers'), ('69', 'Attempts to Revive the SALT II Treaty'), ('70', 'Surrogate Motherhood'), ('71', 'Border Incursions'), ('72', 'Demographic Shifts in the U.S.'), ('73', 'Demographic Shifts across National Boundaries'), ('74', 'Conflicting Policy'), ('75', 'Automation'), ('76', 'U.S. Constitution - Original Intent'), ('77', 'Poaching'), ('78', 'Greenpeace'

### Task 1: Implement and compare lexical IR methods [35 points] ### 

In this task you will implement a number of lexical methods for IR using the **Pyndri** framework. Then you will evaluate these methods on the dataset we have provided using **TREC Eval**.

Use the **Pyndri** framework to get statistics of the documents (term frequency, document frequency, collection frequency; **you are not allowed to use the query functionality of Pyndri**) and implement the following scoring methods in **Python**:

- [TF-IDF](http://nlp.stanford.edu/IR-book/html/htmledition/tf-idf-weighting-1.html) and 
- [BM25](http://nlp.stanford.edu/IR-book/html/htmledition/okapi-bm25-a-non-binary-model-1.html) with k1=1.2 and b=0.75. **[5 points]**
- Language models ([survey](https://drive.google.com/file/d/0B-zklbckv9CHc0c3b245UW90NE0/view))
    - Jelinek-Mercer (explore different values of 𝛌 in the range [0.1, 0.5, 0.9]). **[5 points]**
    - Dirichlet Prior (explore different values of 𝛍 [500, 1000, 1500]). **[5 points]**
    - Absolute discounting (explore different values of 𝛅 in the range [0.1, 0.5, 0.9]). **[5 points]**
    - [Positional Language Models](http://sifaka.cs.uiuc.edu/~ylv2/pub/sigir09-plm.pdf) define a language model for each position of a document, and score a document based on the scores of its PLMs. The PLM is estimated based on propagated counts of words within a document through a proximity-based density function, which both captures proximity heuristics and achieves an effect of “soft” passage retrieval. Implement the PLM, all five kernels, but only the Best position strategy to score documents. Use 𝛔 equal to 50, and Dirichlet smoothing with 𝛍 optimized on the validation set (decide how to optimize this value yourself and motivate your decision in the report). **[10 points]**
    
Implement the above methods and report evaluation measures (on the test set) using the hyper parameter values you optimized on the validation set (also report the values of the hyper parameters). Use TREC Eval to obtain the results and report on `NDCG@10`, Mean Average Precision (`MAP@1000`), `Precision@5` and `Recall@1000`.

For the language models, create plots showing `NDCG@10` with varying values of the parameters. You can do this by chaining small scripts using shell scripting (preferred) or execute trec_eval using Python's `subprocess`.

Compute significance of the results using a [two-tailed paired Student t-test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html) **[5 points]**. Be wary of false rejection of the null hypothesis caused by the [multiple comparisons problem](https://en.wikipedia.org/wiki/Multiple_comparisons_problem). There are multiple ways to mitigate this problem and it is up to you to choose one.

Analyse the results by identifying specific queries where different methods succeed or fail and discuss possible reasons that cause these differences. This is *very important* in order to understand who the different retrieval functions behave.

**NOTE**: Don’t forget to use log computations in your calculations to avoid underflows. 

**IMPORTANT**: You should structure your code around the helper functions we provide below.

In [12]:
with open('./ap_88_89/topics_title', 'r') as f_topics:
    queries = parse_topics([f_topics])

index = pyndri.Index('index/')

num_documents = index.maximum_document() - index.document_base()

dictionary = pyndri.extract_dictionary(index)

tokenized_queries = {
    query_id: [dictionary.translate_token(token)
               for token in index.tokenize(query_string)
               if dictionary.has_token(token)]
    for query_id, query_string in queries.items()}

query_term_ids = set(
    query_term_id
    for query_term_ids in tokenized_queries.values()
    for query_term_id in query_term_ids)

print('Gathering statistics about', len(query_term_ids), 'terms.')

# inverted index creation.

document_lengths = {}
unique_terms_per_document = {}

inverted_index = collections.defaultdict(dict)
collection_frequencies = collections.defaultdict(int)
total_terms = 0

for int_doc_id in range(index.document_base(), index.maximum_document()):
    ext_doc_id, doc_token_ids = index.document(int_doc_id)

    document_bow = collections.Counter(
        token_id for token_id in doc_token_ids
        if token_id > 0)
    document_length = sum(document_bow.values())

    document_lengths[int_doc_id] = document_length
    total_terms += document_length

    unique_terms_per_document[int_doc_id] = len(document_bow)

    for query_term_id in query_term_ids:
        assert query_term_id is not None

        document_term_frequency = document_bow.get(query_term_id, 0)

        if document_term_frequency == 0:
            continue

        collection_frequencies[query_term_id] += document_term_frequency
        inverted_index[query_term_id][int_doc_id] = document_term_frequency

avg_doc_length = total_terms / num_documents

# print('Inverted index creation took', time.time() - start_time, 'seconds.')

Gathering statistics about 456 terms.


In [114]:
def run_retrieval(model_name, score_fn, params):
    """
    Runs a retrieval method for all the queries and writes the TREC-friendly results in a file.
    
    :param model_name: the name of the model (a string)
    :param score_fn: the scoring function (a function - see below for an example) 
    :param params: params needed for chosen model
    """
    if model_name == 'JM' or model_name == 'DP' or model_name == 'AD':
        params_string = '-'.join(map(str, params))
        run_out_path = '{0}-{1}.run'.format(model_name, params_string)
    else:
        run_out_path = '{}.run'.format(model_name)    
        
    # Don't run twice
    if os.path.exists(run_out_path):
        return

    retrieval_start_time = time.time()

    print('Retrieving using', model_name)
    
    data = {}
    for query_id in queries:
        scores = []
        for doc_id in range(index.document_base(), index.maximum_document()):
            ext_doc_id, _ = index.document(doc_id)
            score = 0
            if model_name == 'tfidf':
                for query_token in tokenized_queries[query_id]:
                    score += score_fn(doc_id, query_token, inverted_index, num_documents)
            elif model_name == 'BM25':
                k1 = params[0]
                b = params[1]
                score = score_fn(doc_id, tokenized_queries[query_id], inverted_index, document_lengths, k1, b)
            elif model_name == 'JM' or model_name == 'DP' or model_name == 'AD':
                param = params[0]  
                query_tokens = tokenized_queries[query_id]
                lm = score_fn(doc_id, query_tokens, inverted_index, collection_frequencies, unique_terms_per_document, model_name, param)
                # Calculate P(q|d)
                P_q_d = 1
                for query_token in query_tokens:
                    P_q_d *= lm[query_token]
                score = P_q_d
            elif model_name == 'PLM':
                sigma = params[0]
                mu = params[1]
                query_tokens = tokenized_queries[query_id]
                score = score_fn(doc_id, query_tokens, inverted_index, collection_frequencies, index, sigma, mu)
            scores.append((score, ext_doc_id))
        data[query_id] = scores
    
    # The dictionary data should have the form: query_id --> (document_score, external_doc_id)
    
    with open(run_out_path, 'w') as f_out:
        write_run(
            model_name=model_name,
            data=data,
            out_f=f_out,
            max_objects_per_query=1000)
        
# Any parameters needed for the corresponding score function in the order in which the function takes them as argument
params = [50, 500]
bm25_params = [1.2, 0.75]
run_retrieval('tfidf', tfidf, bm25_params)


Retrieving using BM25


KeyboardInterrupt: 

OrderedDict([('51', 'Airbus Subsidies'),
             ('52', 'South African Sanctions'),
             ('53', 'Leveraged Buyouts'),
             ('54', 'Satellite Launch Contracts'),
             ('55', 'Insider Trading'),
             ('56', 'Prime (Lending) Rate Moves, Predictions'),
             ('57', 'MCI'),
             ('58', 'Rail Strikes'),
             ('59', 'Weather Related Fatalities'),
             ('60', 'Merit-Pay vs. Seniority'),
             ('61', 'Israeli Role in Iran-Contra Affair'),
             ('62', "Military Coups D'etat"),
             ('63', 'Machine Translation'),
             ('64', 'Hostage-Taking'),
             ('65', 'Information Retrieval Systems'),
             ('66', 'Natural Language Processing'),
             ('67', 'Politically Motivated Civil Disturbances'),
             ('68', 'Health Hazards from Fine-Diameter Fibers'),
             ('69', 'Attempts to Revive the SALT II Treaty'),
             ('70', 'Surrogate Motherhood'),
             ('71',

In [13]:
import numpy as np

def evaluate(run_file, qrel_file):
    qrel_location = "ap_88_89/" + qrel_file
    
    map_1000 = !trec_eval/trec_eval -m all_trec -q {qrel_location} {run_file} | grep "map_cut_1000"  
    ndcg_10 = !trec_eval/trec_eval -m all_trec -q {qrel_location} {run_file} | grep "ndcg_cut_10"
    precision_5 = !trec_eval/trec_eval -m all_trec -q {qrel_location} {run_file} | grep "P_5"  
    recall_1000 = !trec_eval/trec_eval -m all_trec -q {qrel_location} {run_file} | grep "recall_1000"  
    
    # Retrieve id and score from trec_eval output
    map_1000 = [(score.split()[1], score.split()[2]) for score in np.array(map_1000)]
    ndcg_10 = [(score.split()[1], score.split()[2]) for score in np.array(ndcg_10)]
    precision_5 = [(score.split()[1], score.split()[2]) for score in np.array(precision_5)]
    recall_1000 = [(score.split()[1], score.split()[2]) for score in np.array(recall_1000)]
   
    return map_1000, ndcg_10, precision_5, recall_1000

map_1000, ndcg_10, precision_5, recall_1000 = evaluate('tfidf.run', 'qrel_validation')

print(map_1000[-1])
print(ndcg_10[-1])
print(precision_5[-1])
print(recall_1000[-1])

('Quit', 'in')
('Quit', 'in')
('Quit', 'in')
('Quit', 'in')


In [14]:
import math

def calculate_tf(document_id, query_term_id, inverted_index):
    try:
        document_term_frequency = inverted_index[query_term_id][document_id]
        log_tf = 1 + math.log(document_term_frequency)
        return log_tf
    except KeyError:
        return 0

def tfidf(int_document_id, query_term_id, inverted_index, N):
    """
    Scoring function for a document and a query term
    
    :param int_document_id: the document id
    :param query_token_id: the query term id (assuming you have split the query to tokens)
    :param document_term_freq: the document term frequency of the query term 
    """
    
    log_tf = calculate_tf(int_document_id, query_term_id, inverted_index)
    document_frequency = len(inverted_index[query_term_id])
    log_idf = math.log(N) - math.log(document_frequency)
    score = log_tf*log_idf
    return score

tfidf(2, 20, inverted_index, num_documents)
# combining the two functions above: 
run_retrieval('tfidf', tfidf)
# TODO implement the rest of the retrieval functions 

# TODO implement tools to help you with the analysis of the results.

NameError: name 'run_retrieval' is not defined

In [110]:
def BM25(document_id, query_ids, inverted_index, document_lengths, k1, b):
    N = len(document_lengths)
    unique_query_terms = set(query_ids)
    l_d = document_lengths[document_id]
    total_length = sum(document_lengths.values())
    l_avg = total_length/N
    BM25 = 0
    for query_term_id in unique_query_terms:
        try:
            log_tf = calculate_tf(document_id, query_term_id, inverted_index)
            document_frequency = len(inverted_index[query_term_id])
            log_idf = math.log(N) - math.log(document_frequency)
            BM25 += ((k1 + 1)*log_tf)/(k1*((1 - b) + b * (l_d/l_avg)) + log_tf) * log_idf
        except KeyError:
            BM25 += 0
    return BM25

BM25(99783, tokenized_queries['90'], inverted_index, document_lengths, 1.2, 0.75)

7.5346740771599805

In [16]:
def JM_smoothing(document_id, query_term_id, collection_frequencies, document_lengths, inverted_index, l):
    log_tf = calculate_tf(document_id, query_term_id, inverted_index)
    tf_C = collection_frequencies[query_term_id]
    log_tf_C = 1 + math.log(tf_C)
    doc_length = document_lengths[document_id]
    len_C = sum(document_lengths.values())
    P_JM = l*(log_tf/doc_length) + (1 - l)*(log_tf_C)/len_C
    return P_JM

l = 0.9
JM_smoothing(99783, 1275, collection_frequencies, document_lengths, inverted_index, l)

0.004776927520830531

In [17]:
def DP_smoothing(document_id, query_term_id, document_lengths, inverted_index, collection_frequencies, mu):
    log_tf = calculate_tf(document_id, query_term_id, inverted_index)
    doc_length = document_lengths[document_id]
    tf_C = collection_frequencies[query_term_id]
    P_w_C = tf_C/sum(collection_frequencies.values())
    P_DP = (doc_length/(doc_length + mu))*(log_tf/doc_length) + (mu/(mu + doc_length))*P_w_C
    return P_DP

mu = 500
DP_smoothing(99783, 1275, document_lengths, inverted_index, collection_frequencies, mu)

0.002994628735132055

In [18]:
def AD_smoothing(document_id, query_term_id, document_lengths, inverted_index, collection_frequencies, unique_terms_per_document, delta):
    log_tf = calculate_tf(document_id, query_term_id, inverted_index)
    doc_length = document_lengths[document_id]
    d_u = unique_terms_per_document[document_id]
    tf_C = collection_frequencies[query_term_id]
    P_w_C = tf_C/sum(collection_frequencies.values())
    P_AD = max(log_tf - delta, 0)/doc_length + ((delta*d_u)/doc_length) * P_w_C
    return P_AD

delta = 0.1
AD_smoothing(99783, 1275, document_lengths, inverted_index, collection_frequencies, unique_terms_per_document, delta)

0.0051056100822080355

In [19]:
from collections import defaultdict

def makeLM(document_id, query_ids, inverted_index, collection_frequencies, unique_terms_per_document, smoothing, param):
    doc_length = document_lengths[document_id]
    len_C = sum(document_lengths.values())
    prob_query = defaultdict(float)
    for query_id in query_ids:
        log_tf = calculate_tf(document_id, query_id, inverted_index)
        tf_C = collection_frequencies[query_id]
        log_tf_C = 1 + math.log(tf_C)
        P_w_C = tf_C/sum(collection_frequencies.values())
        if smoothing == "AD":
            d_u = unique_terms_per_document[document_id]
            P_w_C = tf_C/sum(collection_frequencies.values())
            prob_query[query_id] = max(log_tf - param, 0)/doc_length + ((param*d_u)/doc_length) * P_w_C
        elif smoothing == "DP":
            P_w_C = tf_C/sum(collection_frequencies.values())
            prob_query[query_id] = (doc_length/(doc_length + param))*(log_tf/doc_length) + (param/(param + doc_length))*P_w_C
        elif smoothing == "JM":
            prob_query[query_id] = param*(log_tf/doc_length) + (1 - param)*(log_tf_C)/len_C
    return prob_query

smoothing = "DP"
param = 0.1
doc_id = 99783
LM = makeLM(doc_id, query_term_ids, inverted_index, collection_frequencies, unique_terms_per_document, smoothing, param)


In [20]:
def TFPP(document_id, word_id, inverted_index, tokens, sigma, j):
    tf_sum = 0
    Z_i = 0
    for i in range(len(tokens)):
        if tokens[i] == word_id:
            tf_i = 1
        else:
            tf_i = 0
        kernel = Gaussian(j, i, sigma)
        Z_i += kernel
        tf_sum += tf_i*kernel
    return tf_sum, Z_i

def LMPP(document_id, word_id, inverted_index, collection_frequencies, index, sigma, j, mu):
    tokens = index.document(document_id)[1]
    tf_C = collection_frequencies[word_id]
    P_w_C = tf_C/sum(collection_frequencies.values())
    tf_w, Z_i = TFPP(document_id, word_id, inverted_index, tokens, sigma, j)    
    P_w_D = (tf_w + mu*P_w_C)/(Z_i + mu)
    return P_w_D

def Gaussian(i, j, sigma):
    return math.exp((-(i - j)**2)/(2*sigma**2))

def Triangle(i, j, sigma):
    if abs(i - j) <= sigma:
        return 1 - abs(i - j)/sigma
    else:
        return 0

def Cosine(i, j, sigma):
    if abs(i - j) <= sigma:
        return 0.5*(1 + math.cos((abs((i - j) * math.pi))/sigma))
    else:
        return 0

def Circle(i, j, sigma):
    if abs(i - j) <= sigma:
        return math.sqrt((1 - ((abs(i - j))/sigma)))
    else:
        return 0
    
def Passage(i, j, sigma):
    if abs(i - j) <= sigma:
        return 1
    else:
        return 0

mu = 500
j = 2
sigma = 50
doc_id = 99783
term_id = 1275
LMPP(doc_id, term_id, inverted_index, collection_frequencies, index, sigma, j, mu)

0.00200784812289836

In [21]:
with open("ap_88_89/qrel_test") as f:
    test_set = f.readlines()
with open("ap_88_89/qrel_validation") as f:
    val_set = f.readlines()


In [22]:
from collections import Counter

def PLM_score(document_id, query_ids, inverted_index, collection_frequencies, index, sigma, j, mu):
    score = 0
    query_counter = Counter(query_ids)
    unique_tokens = set(index.document(document_id)[1])
    unique_query_ids = set(query_ids)
    # Only calculate P_w_Q for words in query, for other words P_w_Q = 0
    for word_id in unique_query_ids:
        P_w_Q = query_counter[word_id]/len(query_ids)
        P_w_D = LMPP(document_id, word_id, inverted_index, collection_frequencies, index, sigma, j, mu)
        if P_w_D != 0:
            score += P_w_Q*math.log(P_w_Q/P_w_D)
    return -score
        
def best_position_strategy(document_id, query_ids, inverted_index, collection_frequencies, index, sigma, mu):
    tokens = index.document(document_id)[1]
    highest_score = PLM_score(document_id, query_ids, inverted_index, collection_frequencies, index, sigma, 0, mu)
    for j in range(1, len(tokens)): 
        score = PLM_score(document_id, query_ids, inverted_index, collection_frequencies, index, sigma, j, mu)
        if score > highest_score:
            highest_score = score
    return highest_score

# print(index.document(doc_id)[1])
query = [5063, 151114, 4414, 4160, 78152]
best_position_strategy(doc_id, query, inverted_index, collection_frequencies, index, sigma, mu)

-4.723197087353424

### Task 2: Latent Semantic Models (LSMs) [15 points] ###

In this task you will experiment with applying distributional semantics methods ([LSI](http://lsa3.colorado.edu/papers/JASIS.lsi.90.pdf) **[5 points]** and [LDA](https://www.cs.princeton.edu/~blei/papers/BleiNgJordan2003.pdf) **[5 points]**) for retrieval.

You do not need to implement LSI or LDA on your own. Instead, you can use [gensim](http://radimrehurek.com/gensim/index.html). An example on how to integrate Pyndri with Gensim for word2vec can be found [here](https://github.com/cvangysel/pyndri/blob/master/examples/word2vec.py). For the remaining latent vector space models, you will need to implement connector classes (such as `IndriSentences`) by yourself.

In order to use a latent semantic model for retrieval, you need to:
   * build a representation of the query **q**,
   * build a representation of the document **d**,
   * calculate the similarity between **q** and **d** (e.g., cosine similarity, KL-divergence).
     
The exact implementation here depends on the latent semantic model you are using. 
   
Each of these LSMs come with various hyperparameters to tune. Make a choice on the parameters, and explicitly mention the reasons that led you to these decisions. You can use the validation set to optimize hyper parameters you see fit; motivate your decisions. In addition, mention clearly how the query/document representations were constructed for each LSM and explain your choices.

In this experiment, you will first obtain an initial top-1000 ranking for each query using TF-IDF in **Task 1**, and then re-rank the documents using the LSMs. Use TREC Eval to obtain the results and report on `NDCG@10`, Mean Average Precision (`MAP@1000`), `Precision@5` and `Recall@1000`.

Perform significance testing **[5 points]** (similar as in Task 1) in the class of semantic matching methods.

### Task 3:  Word embeddings for ranking [20 points] (open-ended) ###

First create word embeddings on the corpus we provided using [word2vec](http://arxiv.org/abs/1411.2738) -- [gensim implementation](https://radimrehurek.com/gensim/models/word2vec.html). You should extract the indexed documents using pyndri and provide them to gensim for training a model (see example [here](https://github.com/nickvosk/pyndri/blob/master/examples/word2vec.py)).
   
This is an open-ended task. It is left up you to decide how you will combine word embeddings to derive query and document representations. Note that since we provide the implementation for training word2vec, you will be graded based on your creativity on combining word embeddings for building query and document representations.

Note: If you want to experiment with pre-trained word embeddings on a different corpus, you can use the word embeddings we provide alongside the assignment (./data/reduced_vectors_google.txt.tar.gz). These are the [google word2vec word embeddings](https://code.google.com/archive/p/word2vec/), reduced to only the words that appear in the document collection we use in this assignment.

In [40]:
import gensim
import pyndri.compat

def train_word2vec(corpus):
    
    word2vec_init = gensim.models.Word2Vec(
    size=300,  # Embedding size
    window=5,  # One-sided window size
    sg=True,  # Skip-gram.
    min_count=5,  # Minimum word frequency.
    sample=1e-3,  # Sub-sample threshold.
    hs=False,  # Hierarchical softmax.
    negative=10,  # Number of negative examples.
    iter=1,  # Number of iterations.
    workers=8,  # Number of workers.
)
    word2vec_init.build_vocab(corpus, trim_rule=None)

    models = [word2vec_init]
    model = copy.deepcopy(models[-1])
    model.train(corpus, total_examples=model.corpus_count, epochs=model.iter)

    models.append(model)
    return models
    

corpus = pyndri.compat.IndriSentences(index, dictionary)
models = train_word2vec(corpus)



In [61]:
print(collection_frequencies)

defaultdict(<class 'int'>, {78152: 0, 1: 171350, 3076: 2596, 2053: 4158, 8: 119771, 1033: 8094, 11: 96612, 13: 84131, 1042: 8015, 19: 73013, 20: 71782, 21: 69256, 66582: 353, 1056: 7934, 33: 56945, 35: 55709, 38: 54180, 39: 53986, 2088: 4081, 45097: 965, 690: 11287, 1070: 7819, 178525: 14, 160909: 158, 2097: 4064, 54: 46470, 55: 46166, 57: 45936, 4160: 0, 68: 41803, 69: 41350, 72: 40954, 2121: 4021, 1098: 7709, 75: 40696, 5197: 1308, 78: 40331, 2127: 4010, 80: 40019, 2129: 4004, 82: 39433, 1891: 4572, 85: 38950, 163926: 121, 65624: 747, 97318: 133, 92: 37135, 94: 36935, 97: 36376, 1122: 7543, 235195: 285, 237310: 17, 3173: 2501, 119910: 46, 140391: 737, 2152: 3961, 3177: 2498, 107: 34626, 4204: 1746, 110: 34232, 153960: 2, 114: 33910, 118: 33171, 119: 33025, 218232: 734, 1145: 7379, 231615: 234, 128: 32340, 2177: 3884, 2178: 3883, 2181: 3871, 163974: 574, 2183: 3869, 1160: 7310, 1161: 7298, 138: 31429, 140: 31256, 4072: 1820, 142: 30861, 227473: 220, 254101: 175, 159894: 536, 2199: 383

In [105]:
def text_representation(word2vec_model, tokens, N):
    tokens = tokens.split()
    token_counter = Counter(tokens)
    embedded_tokens = [word2vec_model.wv[token.lower()] for token in tokens]
    token_weights = []
    for i in range(len(tokens)):
        token_id = token2id[tokens[i]]
        document_frequency = len(inverted_index[token_id])
        log_idf = math.log(N) - math.log(document_frequency)
        term_frequency = token_counter[tokens[i]]
        log_tf = 1 + math.log(term_frequency)
        token_weights.append(log_tf*log_idf)
    avg_token_weights = np.array(token_weights)/sum(token_weights)
    representation = []
    for i in range(len(tokens)):
        representation.append(embedded_tokens[i]*avg_token_weights[i])
    return sum(representation)
text_representation(models[1], queries['161'].lower(), num_documents)


array([ 3.42297703e-02,  1.79504812e-01, -1.89787671e-01,  3.73626500e-02,
       -5.40211737e-01,  9.65279713e-02, -1.01969287e-01,  1.76160365e-01,
       -6.62141368e-02, -4.67654288e-01, -9.28428676e-03,  1.57016501e-01,
        4.53330502e-02, -8.76431540e-02,  6.48423582e-02,  1.01889908e-01,
        1.20397657e-04,  3.52838039e-01,  4.03705239e-01,  4.94697303e-01,
        3.83278966e-01, -1.69290170e-01, -1.04092285e-01,  1.09430090e-01,
        1.53636068e-01,  1.82842821e-01,  9.32084545e-02,  5.82165569e-02,
        4.94382158e-02,  2.98161715e-01,  9.26617384e-02, -1.94016263e-01,
       -1.27619639e-01, -1.24728121e-02, -6.97756559e-03,  2.01732069e-02,
       -5.13505220e-01,  3.66343766e-01,  4.70881835e-02,  1.39084443e-01,
        6.20896935e-01,  1.99873522e-02,  3.26636672e-01,  1.68139845e-01,
        2.87087351e-01,  3.04125249e-03,  9.81723815e-02, -1.31375819e-01,
       -3.22964638e-01, -7.24940598e-02,  4.24066216e-01,  5.52364171e-01,
       -1.52556583e-01,  

In [108]:
query = 'acid rain'
query_vector = text_representation(models[1], query, num_documents)

In [109]:
models[1].wv.similar_by_vector(query_vector)

[('acid', 0.9081491231918335),
 ('rain', 0.8299046754837036),
 ('nitric', 0.72994464635849),
 ('sulfur', 0.7215514183044434),
 ('hydrochloric', 0.6968904733657837),
 ('sulfuric', 0.6951553821563721),
 ('dioxide', 0.6951542496681213),
 ('pollutants', 0.6905497908592224),
 ('oxides', 0.6879680156707764),
 ('acidity', 0.6878735423088074)]

### Task 4: Learning to rank (LTR) [15 points] (open-ended) ###

In this task you will get an introduction into learning to rank for information retrieval.

You can explore different ways for devising features for the model. Obviously, you can use the retrieval methods you implemented in Task 1, Task 2 and Task 3 as features. Think about other features you can use (e.g. query/document length). Creativity on devising new features and providing motivation for them will be taken into account when grading.

For every query, first create a document candidate set using the top-1000 documents using TF-IDF, and subsequently compute features given a query and a document. Note that the feature values of different retrieval methods are likely to be distributed differently.

You are adviced to start some pointwise learning to rank algorithm e.g. logistic regression, implemented in [scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).
Train your LTR model using 10-fold cross validation on the test set. More advanced learning to rank algorithms will be appreciated when grading.

### Task 4: Write a report [15 points; instant FAIL if not provided] ###

The report should be a PDF file created using the [sigconf ACM template](https://www.acm.org/publications/proceedings-template) and will determine a significant part of your grade.

   * It should explain what you have implemented, motivate your experiments and detail what you expect to learn from them. **[10 points]**
   * Lastly, provide a convincing analysis of your results and conclude the report accordingly. **[10 points]**
      * Do all methods perform similarly on all queries? Why?
      * Is there a single retrieval model that outperforms all other retrieval models (i.e., silver bullet)?
      * ...

**Hand in the report and your self-contained implementation source files.** Only send us the files that matter, organized in a well-documented zip/tgz file with clear instructions on how to reproduce your results. That is, we want to be able to regenerate all your results with minimal effort. You can assume that the index and ground-truth information is present in the same file structure as the one we have provided.
